In [25]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:

# Data Augmentation for training set
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
)

# Load in the training dataset with augmentation applied
train_generator = train_datagen.flow_from_directory(
    '../binary-brain-classification/training',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

# Only rescale the test set
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

# Load in the test dataset with only rescaling
test_generator = test_datagen.flow_from_directory(
    '../binary-brain-classification/testing',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 4845 images belonging to 2 classes.
Found 392 images belonging to 2 classes.


In [27]:
model = keras.Sequential([

    # Define the input shape
    keras.layers.Input(shape=(224,224,3)),
    
    # CNN layer 1
    keras.layers.Conv2D(16, (3,3), activation=None),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2,2)),
    
    # Flatten and classify
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation = 'sigmoid')

])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 222, 222, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 222, 222, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 197136)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 197136)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │       197,137 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,649 (772.07 KB)

 Trainable params: 197,617 (771.94 KB)

 Non-trainable params: 32 (128.00 B)

In [28]:
# Compile the Model
model.compile(
    optimizer = 'adam',    # Popular optimizer
    loss = 'binary_crossentropy',   # Categorical crossentropy for classification tasks
    metrics = (['accuracy'])    # Measure the accuracy
)

In [29]:
# Run with GPU for better performance
with tf.device('/GPU:0'):   
    # Test the model
    history =  model.fit(
        train_generator,    # Traning dataset from our generator
        epochs = 10    # See all the images 10 times
    )

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


152/152 ━━━━━━━━━━━━━━━━━━━━ 13s 79ms/step - accuracy: 0.7363 - loss: 6.8215
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.9597 - loss: 0.2116
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9825 - loss: 0.0984
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.9709 - loss: 0.1335
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9826 - loss: 0.0790
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9840 - loss: 0.0706
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9651 - loss: 0.2520
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9915 - loss: 0.0357
Epoch 9/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9926 - loss: 0.0305
Epoch 10/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9881 - loss: 0.0484


In [30]:
model.evaluate(test_generator)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.8759 - loss: 1.1161


[1.0203384160995483, 0.8979591727256775]

In [31]:
model.save('./models/binary_model.keras')